In [5]:
import numpy as np

import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)

3000


In [42]:
class Day16():

    def __init__(self, data_link):
        data_txt = open(data_link, 'r').read()
        self.board = np.array([[letter for letter in elem] for elem in data_txt.split('\n')[:-1]])
        self.pos = np.argwhere(self.board == 'S')[0]
        self.end_pos = np.argwhere(self.board == 'E')[0]
        self.good_paths = []
        self.shortest_distance = np.array([[[100000]*4 for i in range(len(self.board[0]))] for j in range(len(self.board[0]))])
        self.spots = np.ones_like(self.board)

    def display(self):
        print('\n'.join([''.join(list(line)) for line in self.board]))
        print()

    def get_neigh(self, pos = None):
        if not pos:
            pos = self.pos
        symbol = {(1, 0):'v', (-1, 0):'^', (0, 1):'>', (0, -1):'<'}
        return [(pos[0]+dir[0], pos[1]+dir[1], symbol[dir]) for dir in [(1, 0), (-1, 0), (0, 1), (0, -1)] if self.board[pos[0]+dir[0], pos[1]+dir[1]] in ['.', 'E']]

    def simplify(self, pos):
        neighs_pt = [(pos[0]+dir[0], pos[1]+dir[1]) for dir in [(1, 0), (-1, 0), (0, 1), (0, -1)] if self.board[pos[0]+dir[0], pos[1]+dir[1]] == '.']
        if len(neighs_pt) == 1:
            self.board[pos[0], pos[1]] = '#'
            self.simplify(neighs_pt[0])
        if len(neighs_pt) == 0:
            self.board[pos[0], pos[1]] = '#'
        return self

    def simplify_board(self):
        dead_ends = []
        for pos in np.argwhere(self.board == '.'):
            neighs_pt = [(pos[0]+dir[0], pos[1]+dir[1]) for dir in [(1, 0), (-1, 0), (0, 1), (0, -1)] if self.board[pos[0]+dir[0], pos[1]+dir[1]] in ['.', 'E', 'S']]
            if len(neighs_pt) <= 1:
                dead_ends.append(pos)
        for end in dead_ends:
            self.simplify(end)
        return self

    def solve(self, pos = None, state = None, dist = None):

        if pos == state == dist == None :
            pos = self.pos
            state = '>'
            dist = 0

        states = {'v':0, '^':1, '>':2, '<':3}

        if self.board[pos[0], pos[1]] == 'E':
            self.shortest_distance[pos[0], pos[1], states[state]] = dist
            return True

        neighs = self.get_neigh((pos[0], pos[1]))

        if not neighs:
            return False

        shortest = self.shortest_distance[pos[0], pos[1], states[state]]

        if shortest >= 0 and dist >= shortest:
            return False
        else:
            self.shortest_distance[pos[0], pos[1], states[state]] = dist
            for neigh in neighs:
                if neigh[2] == state:
                    self.solve((neigh[0], neigh[1]), neigh[2], dist + 1)
                else:
                    self.solve((neigh[0], neigh[1]), neigh[2], dist + 1001)
        return True

    def best_score(self):
        return min([self.shortest_distance[1, -2, i] for i in range(4) if self.shortest_distance[1, -2, i] > 0])

    def on_best(self, pos = None, state = None):
        states_rev = {0:'v', 1:'^', 2:'>', 3:'<'}
        states = {'v':0, '^':1, '>':2, '<':3}

        if not pos or not state:
            pos = (1, -2)
            state = states_rev[np.argmin(self.shortest_distance[pos[0], pos[1]])]
            self.spots[-2, 1] = '.'

        self.spots[pos[0], pos[1]] = '.'

        dist = np.min(self.shortest_distance[pos[0], pos[1]])

        neighs = self.get_neigh(pos)
        if not neighs:
            return False
        for neigh in neighs:
            dist_neigh = np.min(self.shortest_distance[neigh[0], neigh[1]])
            arg_dist = np.argmin(self.shortest_distance[neigh[0], neigh[1]])
            if dist_neigh%1000 - dist%1000 == -1:
                self.on_best((neigh[0], neigh[1]), state)

    def nb_bests(self):
        self.simplify_board()
        self.solve()
        self.on_best()
        return len(np.argwhere(self.spots == '.'))


In [43]:
day16_test = Day16('other/day16_test.txt')
day16_test.simplify_board()
day16_test.solve()
day16_test.best_score()

7036

In [38]:
# day16 = Day16('data/day16.txt')
# day16.simplify_board()
# day16.solve()
# day16.best_score()

In [44]:
day16_test = Day16('other/day16_test.txt')
day16_test.nb_bests()

45

In [45]:
day16 = Day16('data/day16.txt')
day16.nb_bests()

515

In [46]:
print('\n'.join([''.join(list(line)) for line in day16.spots]))
# Mon code ne marche pas, je print et je compte à la main.

111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111.........1.1
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111.1111111.1.1
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111.....111.1.1
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111.111.1.1
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111.111.1.1
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111.111.1.1
111111

In [ ]:
26